# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.8075078261961934

Use a flat priors (the default, omitted) for α

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (25 steps)
8.3e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.003 s/step ...done
MCMC, adapting ϵ (100 steps)
7.6e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
6.2e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
6.5e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.4e-5 s/step ...done
MCMC (1000 steps)
6.9e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.338392], -4.92525, 1, AdjacentTurn, 0.924601, 3), NUTS_Transition{Array{Float64,1},Float64}([0.714611], -2.90385, 1, AdjacentTurn, 0.991313, 3), NUTS_Transition{Array{Float64,1},Float64}([0.119063], -3.15935, 1, AdjacentTurn, 0.933952, 3), NUTS_Transition{Array{Float64,1},Float64}([1.38396], -4.40047, 1, AdjacentTurn, 0.789907, 3), NUTS_Transition{Array{Float64,1},Float64}([1.04908], -3.45971, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.748324], -3.00765, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.808523], -2.86286, 1, DoubledTurn, 0.991439, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.825937], -6.09702, 1, AdjacentTurn, 0.504204, 3), NUTS_Transition{Array{Float64,1},Float64}([0.514522], -4.87216, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.704982], -2.80872, 2, DoubledTurn, 0.996395, 3)  …  NUTS_Transit

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

278.40520152797075

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.31 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 34% DoubledTurn => 66%
  depth: 1 => 67% 2 => 33%


check the mean

In [11]:
mean(posterior_α)

0.6384942752991188

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*